In [ ]:
import numpy as np
import json
import os
from matplotlib import pyplot as plt
#from astropy.modeling import models, fitting

import jungfrau_utils as ju
from alvra_tools import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm
from alvra_tools.channels import *
import csv

### Location of the BSREAD file with PSSS data and the file with corrected images

In [ ]:
bsread_file = "/sf/alvra/data/p18389/raw/test_PSSS/run_000015.BSREAD.h5"
images_file = "/sf/alvra/data/p18389/raw/test_PSSS/run_000016.JF06T08V01.h5"

### Load the PSSS data from BSREAD file using alvra_tools

In [ ]:
(Energy, FWHM, X, Y, pulseID) = \
load_PSSS_data_from_scans_pulseID(bsread_file,channel_PSSS_center, 50)

### Load the images and pulse_ids of images with hits using alvra_tools

In [ ]:
(Images, pulseID_img) = load_corr_JF_data(images_file)

### Filter out spectra that don't have corresponding JF images

In [ ]:
X_filtered = []
Y_filtered = []
Energy_filtered = []

# find matching pulseID's
for i in range(0, len(pulseID_img)):
    for j in range(0, len(pulseID)):
        if pulseID_img[i] == pulseID[j]:
            
            X_filtered.append(X[j])
            Y_filtered.append(Y[j])
            en = float(Energy[j])
            Energy_filtered.append(en)
        

X_filtered = np.asarray(X_filtered)
Y_filtered = np.asarray(Y_filtered)
#Energy_filtered = np.asarray(Energy_filtered)

### Plot single spectra

In [ ]:
plt.figure(figsize=(20,10))
#plt.plot(X[0], Y_filtered[0])
plt.plot(X[0], Y[0])
plt.xlabel('Energy [eV]', fontsize=18)
plt.ylabel('Intensity [a.u]', fontsize=18)
plt.show()

In [ ]:
np.savetxt('/das/work/p18/p18389/karol/PSSS/run_0000_16_intensity.txt', (Y),delimiter=",",fmt="%s") 

## Smooth spectra

In [ ]:
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

plt.figure(figsize=(20,10))
plt.plot(X[0], smooth(Y_filtered[0],50))
plt.xlabel('Energy [eV]', fontsize=18)
plt.ylabel('Intensity [a.u]', fontsize=18)
plt.show()

### Fit the smoothed spectra with astopy

In [ ]:
x = X[0]
y = smooth(Y_filtered[0],50)

#g_init = models.Gaussian1D(amplitude=10000., mean=11210, stddev=15.)
g_init = models.Gaussian1D(amplitude=40000., mean=11200, stddev=19.)

fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)

# Plot the data with the best-fit model
plt.figure(figsize=(20,10))
plt.plot(x, y, 'ko')
plt.plot(x, g(x), label='Gaussian')
plt.xlabel('Energy [eV]', fontsize=18)
plt.ylabel('Intensity [a.u]', fontsize=18)
plt.legend(loc=2)
print(g)
print(Energy[8])
print(pulseID[8])

### Fit all single spectra 

In [ ]:
g_init = models.Gaussian1D(amplitude=40000., mean=11200, stddev=19.)
fit_g = fitting.LevMarLSQFitter()

Energy_fit_mean = []
Energy_fit_amp = []
Energy_fit_std = []

for i in range(0,len(Y_filtered)):
    y = smooth(Y_filtered[i],50)
    g = fit_g(g_init, x, y)
    
    Energy_fit_mean.append(g.mean.value)
    Energy_fit_amp.append(g.amplitude.value)
    Energy_fit_std.append(g.stddev.value)

In [ ]:
Energy_fit_mean

### Reshape arrays (that was a bad idea...)

In [ ]:
#Energy_fit_mean = np.reshape(Energy_fit_mean, (-1,1))
#Energy_fit_amp = np.reshape(Energy_fit_amp, (-1,1))
#Energy_fit_std = np.reshape(Energy_fit_std, (-1,1))
#Energy_fit_mean.shape
#Energy_fit_mean

### Save the hdf5 file with images, pulseIDs, energy from PSSS and from the fit

In [ ]:
save_file = '/das/work/p17/p17984/karol/PSSS/run_000096.energy.JF06T08V01.h5'
#save(save_file, data=Images, pulse_id=pulseID_img, energy_psss=Energy_filtered, energy_fit=Energy_fit_filtered)
save(save_file, data=Images, pulse_id=pulseID_img, energy_psss=Energy_filtered, energy_fit=Energy_fit_mean)

### Check what's in the saved file

In [ ]:
jf_file = h5py.File(save_file)
print(jf_file)
#channel_raw_JF_images = "/data/JF06T08V01/data"

en_psss = jf_file['energy_psss']
en_fit = jf_file['energy_fit']

print(en_psss[0:3])
print(en_fit[0:3])